In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
import random
import time
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

OSError: [WinError 1455] The paging file is too small for this operation to complete. Error loading "C:\Users\huzyfa\anaconda3\lib\site-packages\torch\lib\cufft64_10.dll" or one of its dependencies.

In [14]:
df = pd.read_csv("../data/data.csv")
df.head()

,weekday_name,month,leap_year_condition,decade,output,output_year,output_day
0,2,1,0,180,1-1-1800,0,1
1,3,1,0,180,1-1-1801,1,1
2,4,1,0,180,1-1-1802,2,1
3,5,1,0,180,1-1-1803,3,1
4,6,1,1,180,1-1-1804,4,1


**A- training for getting the year**

In [15]:
# separate target column from input columns
x = df[["leap_year_condition", "decade"]]
y = df["output_year"]
x.head()

,leap_year_condition,decade
0,0,180
1,0,180
2,0,180
3,0,180
4,1,180


In [16]:
# splitting the data into training and test sets
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.19, random_state=1, shuffle=True)

In [25]:
# fix random number generation aka regenerate the same random numbers every time (such as weight and bias initialization )
def set_random_seed(seed=7, deterministic=True):
    """Set random seed, for python, numpy, pytorch

    Args:
        seed (int): Seed to be used.
        deterministic (bool): Whether to set the deterministic option for
            CUDNN backend, i.e., set `torch.backends.cudnn.deterministic`
            to True and `torch.backends.cudnn.benchmark` to False.
            Default: True.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False 
        
set_random_seed      

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class network(nn.Module):
    def __init__(self, out_features):
        super().__init__()
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(17, 17)
        self.fc3 = nn.Linear(17, out_features)                
        
    def forward(self, x):
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

NameError: name 'torch' is not defined

In [ ]:
model = network(in_features=1, in_hidden=6, out_features=10).to(device)

# loss and optimizer initializing
criterion = nn.CrossEntropyLoss()
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=3, verbose=True)
writer = SummaryWriter(f"runs")
print(model)

In [ ]:
num_batches = len(1000)
num_epochs = 10
# starting training loop epochs
st = 0
for epoch in range(num_epochs):
    print(f"starting learning rate = {lr} \n number of epochs = {num_epochs} \n number of batches = {num_batches} \n batch_size = {batch_size}")
    progress = tqdm(total=num_batches, leave=True)
    for batch_idx, (data, labels) in progress: 
        
        # convert data to device
        data = data.to(device=device)
        labels = labels.to(device=device)        
        
        # getting prediction and loss
        preds = model(data)
        loss = criterion(preds, labels)        
        
        # back propagation
        optimizer.zero_grad()
        loss.backward()
        
        # optimization step
        optimizer.step()

        progress.set_description(f"epoch [{1+epoch}/{num_epochs}], loss={loss.item():0.4f}")
        progress.set_postfix()
        writer.add_scalar("training_loss", loss, global_step=st)
        writer.add_hparams({"lr":lr, "batch size": batch_size},
                          {"loss":loss})
        writer.add_histogram("loss", model.fc1.weight)
        st+=1